<a href="https://colab.research.google.com/github/mhered/robotica_patito_pycon2025/blob/main/lerobot/training-act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [2]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 40565, done.
remote: Counting objects: 100% (1119/1119), done.
remote: Compressing objects: 100% (571/571), done.
remote: Total 40565 (delta 927), reused 548 (delta 548), pack-reused 39446 (from 4)
Receiving objects: 100% (40565/40565), 170.51 MiB | 14.08 MiB/s, done.
Resolving deltas: 100% (26295/26295), done.
Filtering content: 100% (45/45), 69.03 MiB | 19.23 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-for

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [1]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: spam-mhered (spam-mhered-duckietown) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Start training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=mhered/recording-test`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
# use this the first time only
!cd lerobot && lerobot-train \
--dataset.repo_id=mhered/recording-test \
--policy.type=act \
--output_dir=outputs/train/act_so100_pato_test1 \
--job_name=act_so100_pato_test \
--policy.device=cuda \
--wandb.enable=true \
--policy.repo_id=mhered/my_act

In [ ]:
# use this when resuming from local training_state
!cd lerobot && lerobot-train \
 --policy.push_to_hub=false \
 --config_path=outputs/train/act_so100_pato_test1/checkpoints/last/pretrained_model/train_config.json \
 --resume=true \
 --steps=50000 \
 --save_checkpoint=true \
 --save_freq=5000 \
 --eval_freq=5000 \
 --policy.device=cuda \
 --num_workers=2 \
 --wandb.enable=true \
 --policy.use_amp=true

INFO 2025-10-04 18:45:46 ot_train.py:148 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

In [14]:
# use this instead to warm start from saved Hub weights
!cd lerobot && lerobot-train \
--dataset.repo_id=mhered/recording-test \
--policy.type=act \
--output_dir=outputs/train/act_so100_pato_test1 \
--job_name=act_so100_pato_test \
--policy.device=cuda \
--wandb.enable=true \
--policy.repo_id=mhered/my_act \
--policy.pretrained_path=mhered/my_act \
--steps=4000 \
--save_checkpoint=true \
--save_freq=2000 \
--eval_freq=2000 \
--policy.use_amp=true \
--num_workers=2 \
--policy.push_to_hub=false


INFO 2025-10-04 18:35:39 ot_train.py:148 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [16]:
!git config --global credential.helper store
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [17]:
!apt-get -qq update && apt-get -qq install -y tree
!tree /content/lerobot/outputs/train/act_so100_pato_test1/checkpoints 2>/dev/null



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package tree.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
/content/lerobot/outputs/train/act_so100_pato_test1/checkpoints
├── 002000
│   ├── pretrained_model
│   │   ├── config.json
│   │   ├── model.safetensors
│   │   ├── policy_postprocessor.json
│   │   ├── policy_postprocessor_step_0_unnormalizer_processor.safetensors
│   │   ├── policy_preprocessor.json
│   │   ├── policy_preprocessor_step_3_normalizer_processor.safetensors
│   │   └── train_config.json
│   └── training_state
│       ├── optimizer_param_groups.json
│       ├── optimizer_state.safetensors
│       ├── rng_state.safet

In [ ]:
!huggingface-cli upload mhered/my_act \
/content/lerobot/outputs/train/act_so100_pato_test1/checkpoints/last/pretrained_model \
--repo-type model --commit-message "Upload last checkpoint"

In [ ]:
!python3 -c "from google.colab import runtime; runtime.unassign()"